In [1]:
import sys
sys.path.append('../')
import newtable
import pprint

In [14]:
core = newtable.ConnectCore.open(
    target=':memory:', 
    dialect='sqlite'
)
core

ConnectCore(target=':memory:', dialect='sqlite', engine=Engine(sqlite:///:memory:), metadata=MetaData())

In [18]:
import dataclasses
f = dataclasses.field()
f

Field(name=None,type=None,default=<dataclasses._MISSING_TYPE object at 0x7fddb0b08160>,default_factory=<dataclasses._MISSING_TYPE object at 0x7fddb0b08160>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=None)

In [15]:
@newtable.table_schema
class MyContainer0:
    id: int
    name: str
    age: int

with core.tablemaker() as tmaker:
    tab0 = tmaker.new_table(container_type=MyContainer0)
pprint.pprint(core.inspect_columns('MyContainer0'))

[{'default': None,
  'name': 'age',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()},
 {'default': None,
  'name': 'id',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()},
 {'default': None,
  'name': 'name',
  'nullable': True,
  'primary_key': 0,
  'type': VARCHAR()}]


In [4]:
with core.query() as q:
    q.insert_multi(tab0, [
        {'name': 'Devin J. Cornell', 'age': 50},
        {'name': 'Dorothy Andrews', 'age': 49},
    ])
    print(q.select(tab0).all())

[(50, None, 'Devin J. Cornell'), (49, None, 'Dorothy Andrews')]


In [5]:
with tab0.query() as q:
    q.insert_single(MyContainer0(id=0, name='John Doe', age=30))
    print(q.select())

[MyContainer0(id=None, name='Devin J. Cornell', age=50), MyContainer0(id=None, name='Dorothy Andrews', age=49), MyContainer0(id=0, name='John Doe', age=30)]


In [6]:
# alternately you can use the core.connect() context manager 
# with the doctable factory method constructor
@newtable.table_schema
class MyContainer01:
    id: int
    name: str
    age: int

with core.connect() as conn:
    tab0 = newtable.DocTable.from_container(MyContainer01, core=core)
    conn.commit()
print(core.inspect_table_names())

['MyContainer0']


In [13]:
import datetime
@newtable.table_schema(table_name='mytable1')
class MyContainer1:
    name: str
    age: int
    id: int = newtable.Column(
        column_args=newtable.ColumnArgs(order=0, primary_key=True, autoincrement=True),
    )
    updated: datetime.datetime = newtable.Column(
        column_args=newtable.ColumnArgs(default=datetime.datetime.utcnow),
    )
    added: datetime.datetime = newtable.Column(
        field_args=newtable.FieldArgs(),
        column_args=newtable.ColumnArgs(
            default=datetime.datetime.utcnow, 
            onupdate=datetime.datetime.utcnow
        )
    )

with core.tablemaker() as tmaker:
    tab1 = tmaker.new_table(container_type=MyContainer1, extend_existing=True)
print(core.inspect_columns('mytable1'))
o = MyContainer1(name='John Doe', age=30)
print(o)
with tab1.query() as q:
    q.insert_single(o)
    print(q.select())

[{'name': 'id', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 1}, {'name': 'added', 'type': DATETIME(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'age', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'updated', 'type': DATETIME(), 'nullable': True, 'default': None, 'primary_key': 0}]
MyContainer1(name='John Doe', age=30, id=MISSING, updated=MISSING, added=MISSING)
[MyContainer1(name='John Doe', age=30, id=1, updated=datetime.datetime(2023, 10, 17, 18, 52, 43, 560316), added=datetime.datetime(2023, 10, 17, 18, 52, 43, 560311)), MyContainer1(name='John Doe', age=30, id=2, updated=datetime.datetime(2023, 10, 17, 18, 52, 54, 298972), added=datetime.datetime(2023, 10, 17, 18, 52, 54, 298968)), MyContainer1(name='John Doe', age=30, id=3, updated=datetime.datetime(2023, 10, 17, 18, 53, 10, 99305), added=datetime.datetime(2023,